In [1]:
!pip install gradio

In [4]:
import cv2
import argparse
import gradio as gr

def highlight_face(net, frame, confidence_threshold=0.7):
    frame_opencv_dnn = frame.copy()
    frame_height, frame_width, _ = frame_opencv_dnn.shape
    blob = cv2.dnn.blobFromImage(frame_opencv_dnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections = net.forward()
    face_boxes = []

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > confidence_threshold:
            x1 = int(detections[0, 0, i, 3] * frame_width)
            y1 = int(detections[0, 0, i, 4] * frame_height)
            x2 = int(detections[0, 0, i, 5] * frame_width)
            y2 = int(detections[0, 0, i, 6] * frame_height)
            face_boxes.append([x1, y1, x2, y2])
            cv2.rectangle(frame_opencv_dnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frame_height / 150)), 8)

    return frame_opencv_dnn, face_boxes

def classify_gender_age(frame, model_mean_values, gender_net, age_net, gender_list, age_list, padding=20):
    result_img, face_boxes = highlight_face(face_net, frame)
    for face_box in face_boxes:
        face = frame[max(0, face_box[1] - padding):min(face_box[3] + padding, frame.shape[0] - 1),
               max(0, face_box[0] - padding):min(face_box[2] + padding, frame.shape[1] - 1)]

        blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), model_mean_values, swapRB=False)
        gender_net.setInput(blob)
        gender_preds = gender_net.forward()
        gender = gender_list[gender_preds[0].argmax()]

        age_net.setInput(blob)
        age_preds = age_net.forward()
        age = age_list[age_preds[0].argmax()]

        cv2.putText(result_img, f'{gender}, {age}', (face_box[0], face_box[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)
    
    return result_img

def classify_gender_age_interface(frame):
    return classify_gender_age(frame, MODEL_MEAN_VALUES, gender_net, age_net, gender_list, age_list, padding=20)

face_proto = "opencvFaceDetector.pbtxt"
face_model = "opencvFaceDetectorUint8.pb"
age_proto = "ageDeploy.prototxt"
age_model = "ageNet.caffemodel"
gender_proto = "genderDeploy.prototxt"
gender_model = "genderNet.caffemodel"

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
age_list = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
gender_list = ['Male', 'Female']

face_net = cv2.dnn.readNet(face_model, face_proto)
age_net = cv2.dnn.readNet(age_model, age_proto)
gender_net = cv2.dnn.readNet(gender_model, gender_proto)

# Set up Gradio Interface
iface = gr.Image()
gr.Interface(fn=classify_gender_age_interface, inputs=iface, outputs="image").launch(share=True)

Running on local URL:  http://127.0.0.1:7863
Running on public URL: https://e48988f762cbf50c08.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
